In [1]:
# math stuff
import numpy as np

# tensorflow
import tensorflow as tf

# suppress numpy scientific e-notation
np.set_printoptions(suppress=True)

In [2]:
print(tf.__version__)

1.4.1


In [3]:
# create dummy data similar to what my database query returns
smooth_data = np.float64(np.arange(80).reshape((20, 4)))
print(smooth_data)

[[  0.   1.   2.   3.]
 [  4.   5.   6.   7.]
 [  8.   9.  10.  11.]
 [ 12.  13.  14.  15.]
 [ 16.  17.  18.  19.]
 [ 20.  21.  22.  23.]
 [ 24.  25.  26.  27.]
 [ 28.  29.  30.  31.]
 [ 32.  33.  34.  35.]
 [ 36.  37.  38.  39.]
 [ 40.  41.  42.  43.]
 [ 44.  45.  46.  47.]
 [ 48.  49.  50.  51.]
 [ 52.  53.  54.  55.]
 [ 56.  57.  58.  59.]
 [ 60.  61.  62.  63.]
 [ 64.  65.  66.  67.]
 [ 68.  69.  70.  71.]
 [ 72.  73.  74.  75.]
 [ 76.  77.  78.  79.]]


In [4]:
# set generator input, parameters, tensorflow sos, eos and pad tokens
sequence = smooth_data
stride = 2
observation_length = 4
prediction_length = 2
total_length = observation_length + prediction_length
enc_dec_seq_length = observation_length + 1
padding_length = total_length - (prediction_length + 2) - 1
# sos, eos and pad token values I am sure will not show up in my scaled data
go = 11
pad = 55
eos = 99


# define generator function
def gen_seq():
    
    # compute number of batches to emit
    num_of_batches = round(((len(sequence) - total_length) / stride))
    
    # transform and emit data in batches
    for i in range(0, num_of_batches * stride, stride):
        result = np.array(sequence[i:i + total_length])
        
        # flip array upside down as data is ordered by date desc
        result_flipped = np.flipud(result)
        
        # preparing encoder inputs, adding end of sequence token
        enc_inp = result_flipped[0:observation_length]
        enc_inp = np.lib.pad(enc_inp, ((0,1),(0,0)), 'constant', constant_values=(eos))
        
        # preparing decoder inputs, adding go and end of sequence tokens
        dec_inp = result_flipped[observation_length:total_length]
        dec_inp = np.lib.pad(dec_inp, ((1,0),(0,0)), 'constant', constant_values=(go))
        dec_inp = np.lib.pad(dec_inp, ((0,1),(0,0)), 'constant', constant_values=(eos))
        dec_inp = np.lib.pad(dec_inp, ((0,padding_length),(0,0)), 'constant', constant_values=(eos)) # was pad
        dec_inp = dec_inp[:,0].reshape((observation_length + 1, 1))
        
        # preparing target values, adding end of sequence tokens
        dec_exp_out = result_flipped[observation_length:total_length]
        dec_exp_out = np.lib.pad(dec_exp_out, ((0,1),(0,0)), 'constant', constant_values=(eos))
        dec_exp_out = np.lib.pad(dec_exp_out, ((0,padding_length + 1),(0,0)), 'constant', constant_values=(eos)) # was pad
        dec_exp_out = dec_exp_out[:,0].reshape((observation_length + 1, 1))
        
        # yield results
        yield enc_inp, dec_inp, dec_exp_out

gen = gen_seq()

In [5]:
enc_inp, dec_inp, dec_exp_out = next(gen)
print(enc_inp)
print("")
print(dec_inp)
print("")
print(dec_exp_out)
print("")

[[ 20.  21.  22.  23.]
 [ 16.  17.  18.  19.]
 [ 12.  13.  14.  15.]
 [  8.   9.  10.  11.]
 [ 99.  99.  99.  99.]]

[[ 11.]
 [  4.]
 [  0.]
 [ 99.]
 [ 99.]]

[[  4.]
 [  0.]
 [ 99.]
 [ 99.]
 [ 99.]]



In [6]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

dim_width_enc_inp = len(smooth_data[1])
dim_width_dec_inp = 1
dim_width_dec_exp_out = dim_width_dec_inp
batch_size = 3
# seq length = batch size because time major shape of tensors
seq_length = tf.range(0, batch_size, delta = 1)

# create tensorflow placeholder
enc_inp = tf.placeholder(
    tf.float64, 
    shape=(enc_dec_seq_length, batch_size, dim_width_enc_inp), 
    name="encoder_input")
dec_inp = tf.placeholder(
    tf.float64, 
    shape=(enc_dec_seq_length, batch_size, dim_width_dec_inp), 
    name="decoder_input")
dec_exp_out = tf.placeholder(
    tf.float64, 
    shape=(enc_dec_seq_length, batch_size, dim_width_dec_exp_out), 
    name="target_sequence")

# create tensorflow dataset from generator
dataset = tf.data.Dataset.from_generator(
    gen_seq,
    (tf.float64, 
     tf.float64,
     tf.float64),
    (tf.TensorShape([enc_dec_seq_length, dim_width_enc_inp]), 
     tf.TensorShape([enc_dec_seq_length, dim_width_dec_inp]),
     tf.TensorShape([enc_dec_seq_length, dim_width_dec_exp_out]))
    )

# prefetch 'batch_size' generator outputs
prefetched = dataset.prefetch(batch_size)

# batch prefetched together
batches = prefetched.batch(batch_size)

# create one shot iterator
enc_inp, dec_inp, dec_exp_out = batches.make_one_shot_iterator().get_next()

# convert input to tensors and transpose for time major
enc_inp, dec_inp, dec_exp_out = sess.run([enc_inp, dec_inp, dec_exp_out])
enc_inp = tf.convert_to_tensor(enc_inp, dtype=tf.float64)
enc_inp = tf.transpose(enc_inp, perm = [1, 0, 2])
dec_inp = tf.convert_to_tensor(dec_inp, dtype=tf.float64)
dec_inp = tf.transpose(dec_inp, perm = [1, 0, 2])
dec_exp_out = tf.convert_to_tensor(dec_exp_out, dtype=tf.float64)
dec_exp_out = tf.transpose(dec_exp_out, perm = [1, 0, 2])

# defining layers and number of units for basic lstm cells
enc_layers = 1
enc_num_units = 5
dec_layers = enc_layers
dec_num_units = enc_num_units

# create encoder MultiRNNCells
enc_cells = []
for i in range(enc_layers):
    with tf.variable_scope("rnn_enc") as rnn_enc:
        enc_cells.append(tf.nn.rnn_cell.BasicLSTMCell(enc_num_units))
enc_cell = tf.nn.rnn_cell.MultiRNNCell(enc_cells)

# create decoder MultiRNNCells
dec_cells = []
for i in range(dec_layers):
    with tf.variable_scope("rnn_dec") as rnn_dec:
        dec_cells.append(tf.nn.rnn_cell.BasicLSTMCell(dec_num_units))
dec_cell = tf.nn.rnn_cell.MultiRNNCell(dec_cells)

# create encoder with encoder input
enc_out, enc_state = tf.nn.dynamic_rnn(
    enc_cell, 
    enc_inp,
    dtype = tf.float64,
    sequence_length = seq_length,
    time_major = True)

# create helper with decoder input
helper = tf.contrib.seq2seq.TrainingHelper(
    dec_inp,
    sequence_length = seq_length,
    time_major = True)

# create decoder with helper and encoder state
decoder = tf.contrib.seq2seq.BasicDecoder(
    dec_cell, helper, enc_state)

# dynamic decoding
final_outputs, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(decoder)

# initialize global variables
sess.run(tf.global_variables_initializer())
#print(sess.run(tf.report_uninitialized_variables()))

logits = final_outputs.rnn_output

# print tensor information and data
# logit data seems multiplied by num_units
print("encoder input")
print(enc_inp)
print(enc_inp.eval())
print("99 is the designated end of sequence token")
print("")
print("decoder input")
print(dec_inp)
print(dec_inp.eval())
print("11 is the designated start of sequence token")
print("99 is the designated end of sequence token")
print("")
print("decoder expected output")
print(dec_exp_out)
print(dec_exp_out.eval())
print("99 is the designated end of sequence token")
print("")
print("logits")
print(logits)
print(sess.run(logits))

encoder input
Tensor("transpose:0", shape=(5, 3, 4), dtype=float64)
[[[ 20.  21.  22.  23.]
  [ 28.  29.  30.  31.]
  [ 36.  37.  38.  39.]]

 [[ 16.  17.  18.  19.]
  [ 24.  25.  26.  27.]
  [ 32.  33.  34.  35.]]

 [[ 12.  13.  14.  15.]
  [ 20.  21.  22.  23.]
  [ 28.  29.  30.  31.]]

 [[  8.   9.  10.  11.]
  [ 16.  17.  18.  19.]
  [ 24.  25.  26.  27.]]

 [[ 99.  99.  99.  99.]
  [ 99.  99.  99.  99.]
  [ 99.  99.  99.  99.]]]
99 is the designated end of sequence token

decoder input
Tensor("transpose_1:0", shape=(5, 3, 1), dtype=float64)
[[[ 11.]
  [ 11.]
  [ 11.]]

 [[  4.]
  [ 12.]
  [ 20.]]

 [[  0.]
  [  8.]
  [ 16.]]

 [[ 99.]
  [ 99.]
  [ 99.]]

 [[ 99.]
  [ 99.]
  [ 99.]]]
11 is the designated start of sequence token
99 is the designated end of sequence token

decoder expected output
Tensor("transpose_2:0", shape=(5, 3, 1), dtype=float64)
[[[  4.]
  [ 12.]
  [ 20.]]

 [[  0.]
  [  8.]
  [ 16.]]

 [[ 99.]
  [ 99.]
  [ 99.]]

 [[ 99.]
  [ 99.]
  [ 99.]]

 [[ 99.]
  [ 99.]
